In [93]:
import pandas as pd
import os
import nltk
from nltk.corpus import stopwords
from nltk.corpus import wordnet as wn
from nltk.stem.wordnet import WordNetLemmatizer
import string
from time import time

In [94]:
stopwords = set(stopwords.words('english'))
punctuation = set(string.punctuation)
lemmatize = WordNetLemmatizer()

In [95]:
# bangalore_ward_list = pd.read_csv('BBMP Ward Information with Map View – OpenCity.csv')
# #bangalore_ward_list.info()
# ward_list = list(bangalore_ward_list['Ward Name'].str.lower())
# stopwords.update(ward_list)

In [96]:
'''
Function to do basic text preprocessing
Step 1: Remove stopwords
Step 2: Remove punctuation
Step 3: Lemmatize the words

Input: String
'''

def preprocessing_text(text_input):
    try:
        stopword_removal = " ".join([word for word in text_input.lower().split() if word not in stopwords])
        remove_punctuation = "".join(word for word in stopword_removal if word not in punctuation)
        preprocessed_text = " ".join(lemmatize.lemmatize(word) for word in remove_punctuation.split())
        return preprocessed_text
    except Exception as e:
        print 'Error at {}'.format(text_input)


In [97]:
df_basavanagudi = pd.read_csv('../lad_csv_files/Karnataka-MLA-LAD-Funds-AC-170-BASAVANAGUDI-2013-2018.csv')

In [98]:
df_basavanagudi['Work'] = df_basavanagudi['Work Name Rough Translation (English)']
df_basavanagudi.drop(['Work Name Rough Translation (English)', 'Sl No.'], axis = 1, inplace=True)

In [99]:
df_basavanagudi.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 84 entries, 0 to 83
Data columns (total 3 columns):
Financial Year          84 non-null object
Value (In Rs. Lakhs)    83 non-null float64
Work                    84 non-null object
dtypes: float64(1), object(2)
memory usage: 2.0+ KB


In [100]:
output_text = df_basavanagudi['Work'].apply(preprocessing_text)

In [101]:
output_list = [i.split() for i in output_text]


In [102]:
len(output_text)

84

In [103]:
output_list[0]

['construction', 'school', 'building', 'built', 'gavipuram', 'guttaahalli']

## Convert the word representation to a matrix representation

In [104]:
import gensim

In [105]:
from gensim import corpora

In [106]:
dictionary = corpora.Dictionary(output_list)

In [107]:
print dictionary

Dictionary(421 unique tokens: [u'limited', u'founder', u'existing', u'cooperative', u'railing']...)


In [108]:
document_term_matrix = [dictionary.doc2bow(word) for word in output_list]
corpora.MmCorpus.serialize('corpus.mm', document_term_matrix)
print len(document_term_matrix)
print document_term_matrix[50]

84
[(17, 1), (29, 1), (36, 1), (51, 1), (53, 1), (73, 1), (219, 2), (301, 1), (303, 1)]


In [109]:
start = time()
Lda = gensim.models.ldamodel.LdaModel
ldamodel = Lda(document_term_matrix, num_topics=5, id2word = dictionary, passes=100)
print 'used: {:.2f}s'.format(time()-start)


used: 3.28s


In [113]:
ldamodel.save('topic.model')


In [114]:
print(ldamodel.print_topics(num_topics=5, num_words=4))


[(0, u'0.040*"water" + 0.037*"ward" + 0.021*"subdivision" + 0.016*"hanumanthanagar"'), (1, u'0.049*"building" + 0.037*"school" + 0.031*"ward" + 0.029*"work"'), (2, u'0.037*"ward" + 0.029*"assembly" + 0.026*"constituency" + 0.026*"temple"'), (3, u'0.060*"assembly" + 0.056*"constituency" + 0.052*"ward" + 0.045*"basavanagudi"'), (4, u'0.022*"r" + 0.022*"ward" + 0.021*"building" + 0.018*"sri"')]


In [119]:
import pyLDAvis.gensim
pyLDAvis.enable_notebook()

In [116]:
d = dictionary
c = gensim.corpora.MmCorpus('corpus.mm')
lda = gensim.models.LdaModel.load('topic.model')

In [122]:
data = pyLDAvis.gensim.prepare(lda, c, d)
pyLDAvis.save_html(data, 'lda.html')


/usr/local/lib/python2.7/dist-packages/pyLDAvis/_prepare.py:387: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  topic_term_dists = topic_term_dists.ix[topic_order]
